In [5]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType
from nba_api.stats.endpoints import playbyplay
import time
import datetime
import pandas as pd

In [3]:
# DEFINE FUNCTION HERE
def get_end_of_quarter_data():
    StartTime = datetime.datetime.now()
    # get NBA team information
    nba_teams = teams.get_teams()

    # initialize empty lists to population with team abbreviations and IDs
    abbreviation_list = [0]*len(nba_teams)
    id_list = [0]*len(nba_teams)

    # run a loop to get all of the team IDs
    for i in range(0,len(nba_teams)):
        abbreviation_list[i] = nba_teams[i]['abbreviation']
        id_list[i] = nba_teams[i]['id']
    abbrev_ids = pd.DataFrame(zip(abbreviation_list, id_list))

    # initialize empty lists to get game matchup and game IDs
    matchup = [0]*len(id_list)
    game_id_list = [0]*len(id_list)

    # run a loop to get all of the game matchup information
    for i in range(0,len(id_list)):
        time.sleep(.600) 
        gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=id_list[i],
                            season_nullable=Season.default,
                            season_type_nullable=SeasonType.regular, timeout = 100)  
        games_dict = gamefinder.get_normalized_dict()
        games = games_dict['LeagueGameFinderResults']
        game = games[0]
        game_id = game['GAME_ID']
        game_matchup = game['MATCHUP']

        print(f'Searching through {len(games)} game(s) for the game_id of {game_id} where {game_matchup}') # OPTIONAL: But nice way to Print the games found for current day
        matchup[i] = game_matchup
        game_id_list[i] = game_id

    # set an empty list to store a dataframe with home and away information 
    home_away = [0]*len(matchup)
    # run a loop to only select games with a "vs." This makes sure there are no duplicates, output is a Boolean and will be used to subset with
    for i in range(0,len(matchup)):
        home_away[i] = "vs." in matchup[i]
    # combine the game_id list with the matchup list based on the index from home_away list
    matchup_df = pd.DataFrame(zip(game_id_list,matchup))[home_away]

    # transform game_id and matchup information into nice looking data frame
    matchup_df[['HOME','AWAY']] = matchup_df[1].str.split('vs.',expand=True)
    matchup_df.columns = ['GAME_ID', 'MATCHUP', 'HOME', 'AWAY']

    # reset columns
    matchup_df1 = matchup_df.reset_index(drop=True)

    # create empty data frame to populate with play by play information
    data = pd.DataFrame()
    # run a loop to concat each game's data frame to the previous data frame
    for i in range(0,len(matchup_df1['GAME_ID'])):
        time.sleep(.600) 
        df = playbyplay.PlayByPlay(matchup_df1['GAME_ID'][i]).get_data_frames()[0]
        data = pd.concat([data, df])
    # reset index of final data frame
    game_data = data.reset_index(drop=True)

    # set a counter as a boolean and run a loop to get the description with the word 'End' this will ensure that all the data gathered is end of quarter data.
    counter = [0]*len(game_data["NEUTRALDESCRIPTION"])
    for i in range(0,len(game_data["NEUTRALDESCRIPTION"])):
        counter[i] = ('End') in str(game_data["NEUTRALDESCRIPTION"][i])

    # subset data with this counter information and call it a new dataframe, quarter_data
    quarter_data = game_data[counter]

    # merge data with the matchup data 
    teams_quarter_data = pd.merge(matchup_df1, quarter_data, on='GAME_ID')
    # select only relevant columns
    teams_quarter_data = teams_quarter_data[['MATCHUP','HOME','AWAY', 'PERIOD', 'NEUTRALDESCRIPTION','SCORE','SCOREMARGIN']]

    # write output to an Excel spreadsheet
    teams_quarter_data.to_excel("~/Desktop/3.14.2021.NBA_Games.xlsx")
    EndTime = datetime.datetime.now()
    print(EndTime - StartTime)

In [6]:
get_end_of_quarter_data()

Searching through 39 game(s) for the game_id of 0022000597 where ATL vs. CLE
Searching through 38 game(s) for the game_id of 0022000598 where BOS @ HOU
Searching through 38 game(s) for the game_id of 0022000597 where CLE @ ATL
Searching through 39 game(s) for the game_id of 0022000601 where NOP vs. LAC
Searching through 37 game(s) for the game_id of 0022000600 where CHI vs. TOR
Searching through 37 game(s) for the game_id of 0022000592 where DAL @ DEN
Searching through 38 game(s) for the game_id of 0022000592 where DEN vs. DAL
Searching through 39 game(s) for the game_id of 0022000594 where GSW vs. UTA
Searching through 37 game(s) for the game_id of 0022000598 where HOU vs. BOS
Searching through 40 game(s) for the game_id of 0022000601 where LAC @ NOP
Searching through 38 game(s) for the game_id of 0022000585 where LAL vs. IND
Searching through 39 game(s) for the game_id of 0022000596 where MIA @ ORL
Searching through 38 game(s) for the game_id of 0022000589 where MIL @ WAS
Searching t